# ArcGIS Connector - Wide to Long Transformation

Этот ноутбук:
1. Читает данные из Google Sheets
2. Трансформирует из Wide to Long формат
3. Загружает данные в ArcGIS Feature Layer


In [ ]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


## Шаг 1: Настройка доступа к Google Sheets

**Вариант 1: OAuth (проще для начала)**
- При первом запуске откроется браузер для авторизации
- Разрешите доступ к Google Sheets

**Вариант 2: Service Account**
- Создайте Service Account в Google Cloud Console
- Скачайте JSON ключ и сохраните как `credentials.json`
- Поделитесь Google Sheet с email из Service Account


In [ ]:
# ВАРИАНТ 1: OAuth (откроется браузер для авторизации)
gc = gspread.oauth()

# ВАРИАНТ 2: Service Account (раскомментируйте если используете)
# SCOPE = [
#     'https://www.googleapis.com/auth/spreadsheets',
#     'https://www.googleapis.com/auth/drive'
# ]
# creds = Credentials.from_service_account_file('credentials.json', scopes=SCOPE)
# gc = gspread.authorize(creds)

# URL вашего Google Sheet
SPREADSHEET_URL = 'https://docs.google.com/spreadsheets/d/12846JbH2PwR0wN8eLVnosg4xujw-04gKyyD6RuElc-4/edit?gid=0#gid=0'

# Открываем таблицу
sh = gc.open_by_url(SPREADSHEET_URL)
worksheet = sh.sheet1  # или sh.worksheet('Sheet1')

print(f"Таблица открыта: {sh.title}")


## Шаг 2: Чтение данных из Google Sheets


In [ ]:
# Читаем все данные
data = worksheet.get_all_records()
df = pd.DataFrame(data)

print(f"Загружено строк: {len(df)}")
print(f"Колонки: {list(df.columns)}")
df.head()


## Шаг 3: Трансформация Wide to Long

Преобразуем колонки 'Значення 1' - 'Значення 10' в длинный формат. 
Согласно ТЗ, каждая строка с заполненными значениями должна быть развернута в отдельные строки.


In [ ]:
# Определяем колонки для нормализации
value_columns = [col for col in df.columns if 'Значення' in col or 'value' in col.lower()]

# Колонки, которые остаются (ID колонки)
id_columns = [col for col in df.columns if col not in value_columns and col not in ['long', 'lat']]

print(f"Колонки для нормализации: {value_columns}")
print(f"ID колонки (остаются): {id_columns}")


In [ ]:
# Трансформация Wide to Long
# Создаем список для хранения развернутых строк
expanded_rows = []

for idx, row in df.iterrows():
    # Для каждой строки создаем отдельные строки для каждого заполненного значения
    for value_col in value_columns:
        if pd.notna(row[value_col]) and row[value_col] != '' and row[value_col] != 0:
            # Создаем новую строку
            new_row = row[id_columns + ['long', 'lat']].copy()
            # Добавляем все значения (для соответствия схеме ArcGIS)
            for vc in value_columns:
                new_row[vc] = row[vc] if pd.notna(row[vc]) else 0
            expanded_rows.append(new_row)

# Создаем новый DataFrame
if expanded_rows:
    df_long = pd.DataFrame(expanded_rows)
else:
    # Если нет развернутых строк, используем исходный формат
    df_long = df.copy()

print(f"После трансформации: {len(df_long)} строк")
df_long.head(10)


## Шаг 4: Подготовка данных для ArcGIS

Маппинг полей согласно ТЗ


In [ ]:
# Маппинг полей согласно ТЗ
field_mapping = {
    'd_date': 'Дата',
    't_region': 'Область',
    't_city': 'Місто',
    'i_value_1': 'Значення 1',
    'i_value_2': 'Значення 2',
    'i_value_3': 'Значення 3',
    'i_value_4': 'Значення 4',
    'i_value_5': 'Значення 5',
    'i_value_6': 'Значення 6',
    'i_value_7': 'Значення 7',
    'i_value_8': 'Значення 8',
    'i_value_9': 'Значення 9',
    'i_value_10': 'Значення 10',
    'long': 'long',
    'lat': 'lat'
}

# Переименовываем колонки согласно маппингу
df_final = df_long.copy()
for arcgis_field, source_field in field_mapping.items():
    if source_field in df_final.columns:
        df_final = df_final.rename(columns={source_field: arcgis_field})

# Проверяем наличие обязательных полей
required_fields = ['long', 'lat', 'd_date', 't_region', 't_city']
missing_fields = [f for f in required_fields if f not in df_final.columns]
if missing_fields:
    print(f"ВНИМАНИЕ: Отсутствуют поля: {missing_fields}")
else:
    print("✅ Все обязательные поля присутствуют")

print(f"\nКолонки в финальном DataFrame: {list(df_final.columns)}")
df_final.head()


## Шаг 5: Подключение к ArcGIS

**Варианты доступа (выберите один):**

1. **OAuth через браузер** (рекомендуется) - просто войдите в свой ArcGIS аккаунт
2. **API Key** - если у вас есть ключ
3. **Username/Password** - если знаете URL портала


In [ ]:
# ВАРИАНТ 1: OAuth (откроется браузер для авторизации) - РЕКОМЕНДУЕТСЯ
# Это самый простой способ - просто войдите в свой ArcGIS аккаунт
gis = GIS()  # Откроет браузер для входа

# ВАРИАНТ 2: Если уже авторизованы, можно использовать профиль
# gis = GIS(profile='your_profile_name')

# ВАРИАНТ 3: Прямой вход (если знаете URL вашего портала)
# gis = GIS("https://your-org.maps.arcgis.com", "username", "password")

# ВАРИАНТ 4: API Key (если есть)
# gis = GIS(api_key='YOUR_API_KEY')

print(f"✅ Подключено к: {gis.properties.name}")
print(f"Пользователь: {gis.users.me.username}")


## Шаг 6: Получение Feature Layer по ID


In [ ]:
# ID Feature Layer из ТЗ
FEATURE_LAYER_ID = '6f8eb77db2dc41cc8261eb836e4b35af'

# Получаем item по ID
item = gis.content.get(FEATURE_LAYER_ID)
print(f"✅ Item найден: {item.title}")
print(f"Type: {item.type}")

# Получаем Feature Layer
if hasattr(item, 'layers') and len(item.layers) > 0:
    feature_layer = item.layers[0]
else:
    feature_layer = item

print(f"✅ Feature Layer готов к работе")

# Проверяем схему полей
print("\nПоля в Feature Layer:")
for field in feature_layer.properties.fields:
    print(f"  - {field['name']} ({field['type']})")


## Шаг 7: Подготовка Features для загрузки


In [ ]:
# Функция для создания Feature из строки DataFrame
def create_feature(row):
    # Создаем геометрию (точка) из координат
    try:
        geometry = {
            "x": float(row['long']),
            "y": float(row['lat']),
            "spatialReference": {"wkid": 4326}  # WGS84
        }
    except (ValueError, KeyError) as e:
        print(f"Ошибка координат: {e}")
        return None
    
    # Создаем атрибуты
    attributes = {}
    for field in field_mapping.keys():
        if field in row.index:
            value = row[field]
            # Преобразуем типы данных
            if field.startswith('i_value_'):
                try:
                    attributes[field] = int(float(value)) if pd.notna(value) and value != '' else 0
                except:
                    attributes[field] = 0
            elif field == 'd_date':
                # Преобразуем дату в timestamp (миллисекунды)
                if pd.notna(value) and value != '':
                    try:
                        dt = pd.to_datetime(value)
                        attributes[field] = int(dt.timestamp() * 1000)
                    except:
                        attributes[field] = None
                else:
                    attributes[field] = None
            else:
                attributes[field] = str(value) if pd.notna(value) and value != '' else None
    
    return {
        "geometry": geometry,
        "attributes": attributes
    }

# Создаем список features
features = []
errors = 0

for idx, row in df_final.iterrows():
    try:
        feature = create_feature(row)
        if feature:
            features.append(feature)
    except Exception as e:
        errors += 1
        if errors <= 5:  # Показываем только первые 5 ошибок
            print(f"Ошибка в строке {idx}: {e}")

print(f"✅ Создано {len(features)} features для загрузки")
if errors > 0:
    print(f"⚠️ Ошибок: {errors}")


In [ ]:
# ОПЦИОНАЛЬНО: Очистка существующих данных
# Раскомментируйте следующую строку, если нужно удалить все существующие features
# feature_layer.delete_features(where="1=1")
# print("Существующие данные удалены")

# Добавляем features батчами (по 1000 за раз для надежности)
batch_size = 1000
total_added = 0
total_errors = 0

for i in range(0, len(features), batch_size):
    batch = features[i:i+batch_size]
    try:
        result = feature_layer.edit_features(adds=batch)
        if result['addResults']:
            success_count = sum(1 for r in result['addResults'] if r.get('success', False))
            error_count = len(batch) - success_count
            total_added += success_count
            total_errors += error_count
            print(f"Батч {i//batch_size + 1}: добавлено {success_count}/{len(batch)} features")
            if error_count > 0:
                print(f"  ⚠️ Ошибок в батче: {error_count}")
    except Exception as e:
        print(f"❌ Ошибка при добавлении батча {i//batch_size + 1}: {e}")
        total_errors += len(batch)

print(f"\n✅ Всего добавлено: {total_added} features")
if total_errors > 0:
    print(f"⚠️ Всего ошибок: {total_errors}")


## Шаг 9: Проверка результата


In [ ]:
# Проверяем количество features в слое
query_result = feature_layer.query(where="1=1", return_count_only=True)
print(f"✅ Всего features в слое: {query_result}")

# Получаем несколько примеров
sample = feature_layer.query(where="1=1", return_geometry=True, result_record_count=5)
print("\nПримеры features:")
for i, feature in enumerate(sample.features[:3], 1):
    print(f"\n{i}. Атрибуты:")
    for key, value in feature.attributes.items():
        if value is not None:
            print(f"   {key}: {value}")

print("\n" + "="*50)
print("✅ Данные успешно загружены в ArcGIS Feature Layer!")
print("="*50)
print(f"\nСсылка на карту: {item.url}")
